In [ ]:
# Adapted from pycharm IDE - running python 3.6 - some expected conversion errors

import numpy as np
import cv2
import time


In [ ]:
# Histogram equalization parameters

def equalizeHistColor(frame):
    # equalize the histogram of color image
    img = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)  # convert to HSV
    img[:, :, 2] = cv2.equalizeHist(img[:, :, 2])  # equalize the histogram of the V channel
    return cv2.cvtColor(img, cv2.COLOR_HSV2RGB)  # convert the HSV image back to RGB format


In [ ]:
# Specify file pathway unless in the GoPro_Pi directory

In [ ]:
# What file are we inputting?

Input_V = '003_camera_p3' # Version - example '001' or '054'
Input_Base_Name = '' # ex. '003_camera_p3'
Input_File_Name = Input_Base_Name + Input_V
Input_Format = '.avi'
Input_File_Full_Name = Input_File_Name + Input_Format

In [ ]:
# What file are we outputting?

Output_V = '_001' # # Version - example '001' or '054'
Output_Base_Name = '003_camera_p3'
Output_File_Name = Output_Base_Name + Output_V
Output_Format = '.avi'
Output_File_Full_Name = Output_File_Name + Output_Format # change the file name if needed

# Output file parameter

In [ ]:
## frame dimensions (x,y) & img dimensions (2x,2y) # for resizing drawn frames
scaling_factorx=1
scaling_factory=1
scaling_factor2x=1
scaling_factor2y=1

# Manipulation Variables,
kernelSize = 21
GB_Kernel = 21

# Edge Detection Parameter
parameter1=20
parameter2=60

intApertureSize=1
# Colours
custom_color_list = []
custom_color_list = \
    ["COLOR_BGR2RGB",
     "COLOR_RGB2BGR",
     "COLOR_BGR2GRAY",
     "COLOR_RGB2GRAY",
     "COLOR_BGR2HSV",
     "COLOR_RGB2HSV",
     "COLOR_RGB2HLS",
     "COLOR_BGR2HLS",
     "COLOR_BGR2XYZ",
     "COLOR_RGB2XYZ",
     "COLOR_BGR2Lab",
     "COLOR_RGB2Luv"]

custom_color_type = 1 # 1-12 - look at elements in custom_color_list

# Thresholding 
threshold1=100
threshold2=200

# Contours
color=(255,0,0)
thickness=2
count = 0

In [ ]:
cap = cv2.VideoCapture(Input_File_Full_Name)  # load the video

while (cap.isOpened()):  # play the video by reading frame by frame
    ret, frame = cap.read()
    if ret == True:
        # equalize the histogram of color image - contrast enhancement
        frame1 = equalizeHistColor(frame)

        # Smoothing
        gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (GB_Kernel, GB_Kernel), 0)

        # Thresholding
        # ret, mask = cv2.threshold(blur, threshold1, threshold2, cv2.THRESH_BINARY)
        # ret, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        # ret, mask = cv2.threshold(blur,threshold1, threshold2,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        mask = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
        # mask = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        kernel = np.ones((3, 3), np.uint8)
        mask = cv2.erode(mask, kernel, iterations=7)  # morphology erosion
        mask = cv2.dilate(mask, kernel, iterations=5)  # morphology dilation

        mask_inv = cv2.bitwise_not(mask)
        img = cv2.bitwise_and(frame1, frame1, mask=mask_inv)
        img = cv2.addWeighted(frame1, 0.1, img, 0.9, 0)

        #Contouring - includes a bounding box - takes from previous gray box
        ret, thresh = cv2.threshold(gray, 75, 100, cv2.THRESH_BINARY_INV)
                                                                            #cv2.CHAIN_APPROX_SIMPLE
        img1, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        if len(contours) != 0:
            c = max(contours, key=cv2.contourArea)  # find the largest contour
            x, y, w, h = cv2.boundingRect(c)  # get bounding box of largest contour
            img2=cv2.drawContours(img, c, -1, color, thickness) # draw largest contour
            # img2 = cv2.drawContours(frame, contours, -1, color, thickness)  # draw all contours
            img3 = cv2.rectangle(img2, (x, y), (x + w, y + h), (0, 0, 255), 2)  # draw red bounding box in img
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img3, "label", (x, h), font, 1,(255,255,255),2)

        writer.write(img)  # save the frame into video file
        count += 1
        if count % 1 == 0:
            # cv2.imshow('Original', frame)  # show the original frame
            cv2.imshow('New', img) #show the new frame
        if cv2.waitKey(1)& 0xFF == ord('q'):
            break
    else:
        print("ref != True")
        break
    # When everything done, release the capture
writer.release()
cap.release()
cv2.destroyAllWindows()